In [2]:
import os 
import numpy as np

import argparse
import socket
import time
import sys

import torchvision
from torchvision import transforms
from torchvision.models import resnet50
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import copy

In [3]:
print(torch.__version__)

1.8.1+cu111


In [4]:
parser = argparse.ArgumentParser('argument for training')
# dataset
parser.add_argument('--model', type=str, default='resnet50')

# specify folder
parser.add_argument('--model_path', type=str, default='../checkpoint_0302/ours_style1_best.pth', help='path to save model')
parser.add_argument('--data_root', type=str, default='../../dataset/shape_texture_v2/', help='path to data root')

opt = parser.parse_args(args=[])

In [5]:
# trained classifier를 load해오기
opt.n_class = 20
model = resnet50(pretrained = True)
    
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, opt.n_class)

model.load_state_dict(torch.load(opt.model_path))

<All keys matched successfully>

In [6]:
for param in model.parameters():
    param.requires_grad_(False)
model = model.cuda()

In [7]:
def get_features(image, model):
    # model 레이어별로 feature 뽑아오기 (for content, gram matrix)
    x = image
    features = {}

    x = model.conv1(x)
    x = model.bn1(x)
    x = model.relu(x)
    x = model.maxpool(x)
    i = 0
    
    print(x.shape)

    for name, layer in model.layer1._modules.items():
        x = layer(x)
        i+=1
        features[f"bneck_{name}_{i}"] = x

    for name, layer in model.layer2._modules.items():
        x = layer(x)
        i+=1
        features[f"bneck_{name}_{i}"] = x

    for name, layer in model.layer3._modules.items():
        x = layer(x)
        i+=1
        features[f"bneck_{name}_{i}"] = x

    for name, layer in model.layer4._modules.items():
        x = layer(x)
        i+=1
        features[f"bneck_{name}_{i}"] = x


    return features     

In [8]:
# 생각읋 해보면, 전체 데이터셋에 대해서 할 필요는 없음 -> 데이터 한번 줄이기 (20개의 클래스별로 x 100장씩 -> 2000개씩)
# np.random.seed(10)
# import shutil
# path = '../../dataset/ours_style1_set/train'

# def save_file(class_path, save_path, filenames, start_num, end_num):
#     print(start_num)
#     print(end_num)
#     target_filenames = filenames[start_num:end_num]
#     for target_filename in target_filenames:
#         src = os.path.join(class_path, target_filename)
#         dst = os.path.join(save_path, target_filename)
#         shutil.copy(src, dst)
        
# def split_train_test():
#     for car_class in os.listdir(path):
        
#         car_class_path = os.path.join(path, car_class)
#         car_filenames = os.listdir(car_class_path)
#         train_path = os.path.join('../../dataset/ours_style1_set_100_metric_3/', car_class)

#         if not os.path.exists(train_path):
#             os.makedirs(train_path)

#         save_file(car_class_path, train_path, car_filenames, 0, 10)

# split_train_test()

In [9]:
# get datasets
test_transforms_option = transforms.Compose([
                    transforms.Resize((256, 256)),
                    transforms.ToTensor(),
                    #transforms.Normalize([0.5071, 0.4867, 0.4408], [0.2675, 0.2565, 0.2761])
                ])
test_datasets = torchvision.datasets.ImageFolder(root=opt.data_root, transform = test_transforms_option)
test_loader = torch.utils.data.DataLoader(test_datasets, batch_size = 16, shuffle=False, num_workers = 4)

In [10]:
i = 0
## 레이어별로 feature 뽑아서 저장
image_features_output = dict()

with torch.no_grad() :
    
    for idx, (input, target) in enumerate(test_loader) :
        input = input.float()
        if torch.cuda.is_available():
            input = input.cuda()
            target = target.cuda()
        
        image_features = get_features(input, model)
        
        if i == 0 :
            for key, item in image_features.items():
                image_features_output[key] = item
        else :
            for key, item in image_features.items() : 
                image_features_output[key] = torch.cat([image_features_output[key], image_features[key]])
            
        i += 1

torch.Size([16, 64, 64, 64])
torch.Size([16, 64, 64, 64])
torch.Size([16, 64, 64, 64])
torch.Size([16, 64, 64, 64])
torch.Size([16, 64, 64, 64])
torch.Size([16, 64, 64, 64])
torch.Size([5, 64, 64, 64])


In [11]:
## 어떤식으로 저장되었는지 확인
for key, item in image_features_output.items():
    print(key)
    print(item.shape)
    print()

bneck_0_1
torch.Size([101, 256, 64, 64])

bneck_1_2
torch.Size([101, 256, 64, 64])

bneck_2_3
torch.Size([101, 256, 64, 64])

bneck_0_4
torch.Size([101, 512, 32, 32])

bneck_1_5
torch.Size([101, 512, 32, 32])

bneck_2_6
torch.Size([101, 512, 32, 32])

bneck_3_7
torch.Size([101, 512, 32, 32])

bneck_0_8
torch.Size([101, 1024, 16, 16])

bneck_1_9
torch.Size([101, 1024, 16, 16])

bneck_2_10
torch.Size([101, 1024, 16, 16])

bneck_3_11
torch.Size([101, 1024, 16, 16])

bneck_4_12
torch.Size([101, 1024, 16, 16])

bneck_5_13
torch.Size([101, 1024, 16, 16])

bneck_0_14
torch.Size([101, 2048, 8, 8])

bneck_1_15
torch.Size([101, 2048, 8, 8])

bneck_2_16
torch.Size([101, 2048, 8, 8])



In [12]:
# content matrix는 higher layer에서 뽑아서 지정
content_mat = image_features_output['bneck_4_12'].cpu().detach().numpy()
content_mat.shape

(101, 1024, 16, 16)

In [13]:
def gram_matrix(tensor):
    
    # input 이미지의 값을 텐서로 받아온다
    _, d, h, w = tensor.shape
    
    # multiplying the features for each channel을 위해 reshape
    tensor = tensor.reshape(d, h * w)
    
    # gram matrix
    gram = np.matmul(tensor, tensor.T)
    
    return gram

In [14]:
# lower layer에서 gram matrix를 뽑음 
i = 0

for j in range(content_mat.shape[0]) :
    
    if j % 1000 == 0 :
        print('-- check sample number : ', j)
        
    gram_tmp_output = gram_matrix(image_features_output['bneck_1_2'][j].unsqueeze(0).cpu().detach().numpy())
    gram_tmp_output = np.expand_dims(gram_tmp_output, axis=0)

    if i == 0 :
        gram_output = gram_tmp_output
    else :
        gram_output = np.concatenate((gram_output, gram_tmp_output), axis = 0 )
    i += 1

print(gram_output.shape)

-- check sample number :  0
(101, 256, 256)


In [15]:
content_mat.shape

(101, 1024, 16, 16)

## gram matrix, content matrix cluster 각각 해보기
- for outlier 제거

In [16]:
## content matrix, gram matrix 모두 2d 형태로 변형
gram_output_2d = np.reshape(gram_output, (gram_output.shape[0],(gram_output.shape[1] * gram_output.shape[1] )))
content_output_2d = np.reshape(content_mat, (content_mat.shape[0], (content_mat.shape[1] * content_mat.shape[2] * content_mat.shape[3] )))

print(gram_output_2d.shape)
print(content_output_2d.shape)

(101, 65536)
(101, 262144)


In [17]:
# 파일 이름 저장
file_nm_list = list()

for i in range(len(test_datasets.imgs)) : 
    file_nm = test_datasets.imgs[i][0]
    file_nm_list.append(file_nm)

In [18]:
import pandas as pd 
cluster_df = pd.DataFrame({'file_nm' : file_nm_list})
cluster_df.head()

,file_nm
0,../../dataset/shape_texture_v2/airplane/airpla...
1,../../dataset/shape_texture_v2/airplane/airpla...
2,../../dataset/shape_texture_v2/airplane/airpla...
3,../../dataset/shape_texture_v2/airplane/airpla...
4,../../dataset/shape_texture_v2/airplane/airpla...


In [19]:
# shape에 대한 cluster
from sklearn.cluster import KMeans

k = 10
model = KMeans(n_clusters = k, random_state = 10, max_iter = 1000)
model.fit(content_output_2d)

cluster_df['shape_cluster'] = model.fit_predict(content_output_2d)

In [20]:
cluster_df['shape_cluster'].value_counts()

1    82
0     5
4     4
5     3
8     2
6     1
2     1
3     1
9     1
7     1
Name: shape_cluster, dtype: int64

In [21]:
# outlier는 삭제해볼까
outlier_list = list()

outlier_list.extend(cluster_df[cluster_df['shape_cluster'] == 2]['file_nm'].to_list())
outlier_list.extend(cluster_df[cluster_df['shape_cluster'] == 8]['file_nm'].to_list())
outlier_list.extend(cluster_df[cluster_df['shape_cluster'] == 3]['file_nm'].to_list())
outlier_list.extend(cluster_df[cluster_df['shape_cluster'] == 9]['file_nm'].to_list())
outlier_list.extend(cluster_df[cluster_df['shape_cluster'] == 5]['file_nm'].to_list())
outlier_list.extend(cluster_df[cluster_df['shape_cluster'] == 7]['file_nm'].to_list())
outlier_list.extend(cluster_df[cluster_df['shape_cluster'] == 6]['file_nm'].to_list())

In [22]:
# texture에 대한 cluster
k = 5
model = KMeans(n_clusters = k, random_state = 10, max_iter = 1000)
model.fit(gram_output_2d)

cluster_df['texture_cluster'] = model.fit_predict(gram_output_2d)

In [23]:
cluster_df['texture_cluster'].value_counts()

0    41
2    24
3    17
1    16
4     3
Name: texture_cluster, dtype: int64

In [24]:
# outlier는 삭제해볼까
outlier_texture_list = list()
outlier_texture_list.extend(cluster_df[cluster_df['texture_cluster'] == 4]['file_nm'].to_list())
outlier_texture_list

['../../dataset/shape_texture_v2/airplane/airplane_0795.png',
 '../../dataset/shape_texture_v2/automobile/automobile_4109.png',
 '../../dataset/shape_texture_v2/cat/cat_0659.png']

## distance 기반으로 pair
- (1) shape은 다르고 texture는 같은 pair
    - 동일한 texture에 대해서 구분완료 (stylized의 형태에 따름)
- (2) shape은 같고 texture는 다른 pair 
    - 동일한 shape으로 분류되지는 않지만 (=동일한 클래스는 아님)
    - local feature이 비슷한 shape끼리 매칭되는 것 확인

In [25]:
# shape은 다르고 texture는 같은 것 pair (완료)
texture_pair_list = list()

for i in range(len(cluster_df)):
    tmp_pair_list = list()
    print('############################################## ')
    
    ## pair의 첫번째 img
    print(cluster_df['file_nm'][i])
    tmp_pair_list.append(cluster_df['file_nm'][i])
    
    content_mat_i = content_mat[i]
    gram_mat_i = gram_output[i]
    
    loss_list = list()
    
    ## 전체 이미지들과 shape, texture distance 구해서 계산
    for j in range(len(cluster_df)):
        
        content_mat_j = content_mat[j]
        gram_mat_j = gram_output[j]
        
        # shape, texture loss
        loss_shape = ( content_mat_i - content_mat_j ) ** 2
        loss_texture = ( gram_mat_i - gram_mat_j ) ** 2

        # normalize (texture scale >>> shape scale)
        normalized_shape = loss_shape / np.sqrt(np.sum(loss_shape))
        normalized_texture = loss_texture / np.sqrt(np.sum(loss_texture))
        
        if i == j :
            # 자기 자신은 제외
            loss = 100000000000
        else : 
            loss = np.mean(normalized_texture) ** 2 / np.mean(normalized_shape)
        loss_list.append(loss)
    
    ## distance 제일 작은 것 index
    min_idx = np.argmin(np.array(loss_list))
    
    ## pair의 두번째 이미지
    shape_distance_min = cluster_df['file_nm'][min_idx]#[min_idx+(i+1)]
    tmp_pair_list.append(shape_distance_min)
    
    if len(tmp_pair_list) !=2 : 
        print('check the local lenghts of the list!')
    
    ## 최종적인 Ouput 형태
    texture_pair_list.append(tmp_pair_list)
    print(shape_distance_min)
    print(np.mean(loss_list[min_idx]))
    print()

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_0011.png
../../dataset/shape_texture_v2/frog/frog_1102.png
7064.046933684116

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_0680.png
../../dataset/shape_texture_v2/cat/cat_1210.png
6110.409934840649

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_0721.png


/tmp/ipykernel_1943/2393913110.py:28: RuntimeWarning: invalid value encountered in true_divide
  normalized_shape = loss_shape / np.sqrt(np.sum(loss_shape))
/tmp/ipykernel_1943/2393913110.py:29: RuntimeWarning: invalid value encountered in true_divide
  normalized_texture = loss_texture / np.sqrt(np.sum(loss_texture))


../../dataset/shape_texture_v2/dog/dog_2070.png
9097.255533900374

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_0795.png
../../dataset/shape_texture_v2/automobile/automobile_4109.png
73929.79101330275

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_1098.png
../../dataset/shape_texture_v2/dog/dog_0238.png
8360.131695769356

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_4897.png
../../dataset/shape_texture_v2/dog/dog_0238.png
14454.489388711405

############################################## 
../../dataset/shape_texture_v2/airplane_stylized/airplane_0601_stylized_Ink and wash painting_Ink and wash painting_15469.jpg
../../dataset/shape_texture_v2/ship_stylized/ship_3167_stylized_Ink and wash painting_Ink and wash painting_100664.jpg
1917.2219817749515

############################################## 
../../dataset/shape_texture_v2/airplan

In [26]:
# shape은 같고 texture는 다른 것 pair
correct = 0
shape_pair_list = list()

for i in range(len(cluster_df)):
    if cluster_df['file_nm'][i] in outlier_texture_list  : 
        continue
    else :
        tmp_pair_list = list()
        print('############################################## ')

        ## pair의 첫번째 img
        print(cluster_df['file_nm'][i])
        tmp_pair_list.append(cluster_df['file_nm'][i])

        content_mat_i = content_mat[i]
        gram_mat_i = gram_output[i]

        loss_list = list()

        ## 전체 이미지들과 shape, texture distance 구해서 계산
        for j in range(len(cluster_df)):
            
            if cluster_df['file_nm'][j] in outlier_texture_list : 
                continue 
            else : 
                content_mat_j = content_mat[j]
                gram_mat_j = gram_output[j]

                # shape, texture loss
                loss_shape = ( content_mat_i - content_mat_j ) ** 2
                loss_texture = ( gram_mat_i - gram_mat_j ) ** 2

                # normalize (texture scale >>> shape scale)
                normalized_shape = loss_shape / np.sqrt(np.sum(loss_shape))
                normalized_texture = loss_texture / np.sqrt(np.sum(loss_texture))

                if i == j :
                    # 자기 자신은 제외
                    loss = 1000000000
                else : 
                    loss = np.mean(normalized_shape) / np.mean(normalized_texture) 
                loss_list.append(loss)
                
        if cluster_df['file_nm'][j] in outlier_texture_list : 
            continue 
        else : 
            ## distance 제일 작은 것 index
            min_idx = np.argmin(np.array(loss_list))

            ## pair의 두번째 이미지
            shape_distance_min = cluster_df['file_nm'][min_idx]#[min_idx+(i+1)]


            print(shape_distance_min)
            tmp_pair_list.append(shape_distance_min)
            
            if len(tmp_pair_list) != 2 : 
                print('check the lenghts of the tmp pair list!')
                break
                
            shape_pair_list.append(tmp_pair_list)
            print(np.mean(loss_list[min_idx]))
            print()
            
            # 얼마나 맞추는지 확인
            if 'stlyized' in cluster_df['file_nm'][i] and 'stlyized' not in cluster_df['file_nm'][j]:
                
                if cluster_df['file_nm'][i].split('/')[-2].split('_')[0] == cluster_df['file_nm'][j].split('/')[-2].split('_')[0] :
                    correct += 1
            elif 'stlyized' in cluster_df['file_nm'][j] and 'stlyized' not in cluster_df['file_nm'][i]:
                
                if cluster_df['file_nm'][i].split('/')[-2].split('_')[0] == cluster_df['file_nm'][j].split('/')[-2].split('_')[0] :
                    correct += 1

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_0011.png
../../dataset/shape_texture_v2/automobile_stylized/automobile_0175_stylized_Ink and wash painting_Ink and wash painting_12129.jpg
0.00032762974

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_0680.png
../../dataset/shape_texture_v2/automobile_stylized/automobile_1039_stylized_Ink and wash painting_Ink and wash painting_13532.jpg
0.00016854418

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_0721.png


/tmp/ipykernel_1943/1725125863.py:35: RuntimeWarning: invalid value encountered in true_divide
  normalized_shape = loss_shape / np.sqrt(np.sum(loss_shape))
/tmp/ipykernel_1943/1725125863.py:36: RuntimeWarning: invalid value encountered in true_divide
  normalized_texture = loss_texture / np.sqrt(np.sum(loss_texture))


../../dataset/shape_texture_v2/deer_stylized/deer_4985_stylized_Ink and wash painting_Ink and wash painting_18808.jpg
0.00015663767

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_1098.png
../../dataset/shape_texture_v2/automobile_stylized/automobile_1039_stylized_Ink and wash painting_Ink and wash painting_13532.jpg
0.0001558402

############################################## 
../../dataset/shape_texture_v2/airplane/airplane_4897.png
../../dataset/shape_texture_v2/bird/bird_0046.png
0.00017369872

############################################## 
../../dataset/shape_texture_v2/airplane_stylized/airplane_0601_stylized_Ink and wash painting_Ink and wash painting_15469.jpg
../../dataset/shape_texture_v2/automobile_stylized/automobile_1039_stylized_Ink and wash painting_Ink and wash painting_13532.jpg
0.00018199769

############################################## 
../../dataset/shape_texture_v2/airplane_stylized/airplane_0671_stylized_Ink and

In [27]:
# pair의 형태 (이중 리스트) -> 형태, Lengths 확인

## shape pair list
print(len(shape_pair_list))
print(shape_pair_list[0])

## texture pair list
print(len(texture_pair_list))
print(texture_pair_list[0])

98
['../../dataset/shape_texture_v2/airplane/airplane_0011.png', '../../dataset/shape_texture_v2/automobile_stylized/automobile_0175_stylized_Ink and wash painting_Ink and wash painting_12129.jpg']
101
['../../dataset/shape_texture_v2/airplane/airplane_0011.png', '../../dataset/shape_texture_v2/frog/frog_1102.png']


### Mutual Information

In [28]:
# 데이터를 Pair로 묶었을 때, mutual information 뽑는 부분 작성
import os 
import config
from utils import *
import csv
from argparse import ArgumentParser
import json

In [29]:
# get config
parser = argparse.ArgumentParser(description='Dimension estimation')
parser.add_argument('--save_dir', default='dim_outputs/cifar10/', help="dataset to use for dim estimation")
parser.add_argument('--model', default='resnet50', help="model to do dimension estimation on")
parser.add_argument('--pretrained', default=True, help="whether pre-trained flag is true")
parser.add_argument('--n_factors', default=3, help="number of factors (including residual)")
parser.add_argument('--residual_index', default=2, help="index of residual factor (usually last)")
parser.add_argument('--batch_size', default=4, help="batch size during evaluation")
parser.add_argument('--image_size', default=256, type=int, help="image size during evaluation")
parser.add_argument('--num_workers', default=4, help="number of CPU threads")
parser.add_argument('--device', default='cuda:0', help="gpu id")
parser.add_argument('--dataset', default='stylized_cifar10')
parser.add_argument('--gp', default=None, type=str, metavar='POOL',
                        help='Global pool type, one of (fast, avg, max, avgmax, avgmaxc). Model default if None.')
parser.add_argument('--torchscript', dest='torchscript', action='store_true',
                        help='convert model torchscript for inference')

#args = parser.parse_args()
args = parser.parse_args(args=[])

In [30]:
if not os.path.exists(args.save_dir) : 
    os.mkdir(args.save_dir)

In [31]:
# get trained classifier
print('-- Loading trained classifier')
model = get_model(args)

device = args.device
model.cuda(device)
model.eval()

-- Loading trained classifier


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [32]:
args.n_class = 20
args.model_path = '../checkpoint_0302/ours_style1_best.pth'
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, args.n_class)

model.load_state_dict(torch.load(args.model_path))

<All keys matched successfully>

In [33]:
# get dataset
print('-- Get dataset')
dataloader = get_dataloader(args, shape_pair_list, texture_pair_list)

-- Get dataset


In [34]:
# factor list를 생성 (model의 output feature를 저장할 부분)
factor_list = list()
output_dict = {'example1' : list(), 
               'example2' : list() }

In [35]:
print('-- Start preprocessing to estimate dim')
for i, (factor, example1, example2) in enumerate(dataloader) : 
    
    example1, example2 = example1.cuda(device), example2.cuda(device)
    
    # pass images through model and get distribution mean value
    output1 = model(example1).mode()[0]
    output2 = model(example2).mode()[0]
    
    # factor과 output 저장
    factor_list.append(factor.detach().cpu().numpy() )
    output_dict['example1'].append(output1.detach().cpu().numpy() )
    output_dict['example2'].append(output2.detach().cpu().numpy() )
    
    if i % 10 == 0 : 
        print(f'-- check Iter {i}/{len(dataloader)}')
    
print('-- Finished processing examples ...')
print('-- Starting Din estimation ... ')

-- Start preprocessing to estimate dim
-- check Iter 0/25
-- Finished processing examples ...
-- Starting Din estimation ... 


In [46]:
def dim_est(output_dict, factor_list, args) :
    
    # step1. get vectors from the input pair images
    za = np.concatenate(output_dict['example1'])
    zb = np.concatenate(output_dict['example2'])
    factors = np.concatenate(factor_list)
    
    
    # step2. factor(shape, texture)별로 다시 계산 for score
    za_by_factor = dict()
    zb_by_factor = dict()
    mean_by_factor = dict()
    score_by_factor = dict()
    individual_scores = dict()
    
    zall = np.concatenate([za, zb], 0)
    mean = np.mean(zall, 0, keepdims = True)
    var = np.sum(np.mean((zall-mean)*(zall-mean), 0))
    
    ## step 2-1. 각 factor별로 돌면서 다시 계산
    for f in range(args.n_factors) : 
        if f != args.residual_index : 
            indices = np.where(factors == f)[0]
            za_by_factor[f] = za[indices]
            zb_by_factor[f] = zb[indices]
            mean_by_factor[f] = 0.5*(np.mean(za_by_factor[f], 0, keepdims=True)+np.mean(zb_by_factor[f], 0, keepdims=True))
            
            score_by_factor[f] = np.sum(np.mean((za_by_factor[f]-mean_by_factor[f])*(zb_by_factor[f]-mean_by_factor[f]), 0))
            score_by_factor[f] = score_by_factor[f]/var
            
            idv = np.mean((za_by_factor[f]-mean_by_factor[f])*(zb_by_factor[f]-mean_by_factor[f]), 0)/var
            individual_scores[f] = idv
        
        else : 
            # residual factor에 해당되는 경우
            score_by_factor[f] = 1.0
    
    scores = np.array([score_by_factor[f] for f in range(args.n_factors) ])
    
    # step3. softmax output (shape(0) or texture(1))
    m = np.max(scores)
    e = np.exp(scores-m)
    softmaxed = e/np.sum(e)
    
    dim = za.shape[1]
    dims = [int(s*dim) for s in softmaxed]
    
    dims[-1] = dim - sum(dims[:-1])
    dims_percent = dims.copy()
    
    for i in range(len(dims)) : 
        dims_percent[i] = round(100*(dims[i] / sum(dims,)), 1)
    return dims, dims_percent

In [48]:
dims, dims_percent = dim_est(output_dict, factor_list, args)
print(" >>> Estimated factor dimensionalities: {}".format(dims))
print(" >>> Ratio to total dimensions: {}".format(dims_percent))
print('Saving results to {}'.format(args.save_dir))

 >>> Estimated factor dimensionalities: [335, 405, 1308]
 >>> Ratio to total dimensions: [16.4, 19.8, 63.9]
Saving results to dim_outputs/cifar10/


In [ ]:
# 0; shape
# 1; texture
# 2; residual